In [1]:
import os
import urllib.request

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import wandb
from wandb.keras import WandbCallback

from tensorflow import keras
from tensorflow.keras import layers

# ---------------------------------------------------
# 1. W&B login and config
# ---------------------------------------------------
wandb.login()

config = {
    "project_name": "Lab1-visualize-models",
    "run_name": "keras-mlp-dermatology",
    "test_size": 0.2,
    "random_state": 42,
    "batch_size": 32,
    "epochs": 60,
    "learning_rate": 1e-3,
    "hidden_units": [128, 64],
    "dropout_rate": 0.25
}

run = wandb.init(
    project=config["project_name"],
    name=config["run_name"],
    config=config
)

# ---------------------------------------------------
# 2. Download and load the Dermatology dataset
# ---------------------------------------------------
data_url = "https://archive.ics.uci.edu/ml/machine-learning-databases/dermatology/dermatology.data"
data_path = "dermatology.data"

if not os.path.exists(data_path):
    urllib.request.urlretrieve(data_url, data_path)

# The dataset has 34 feature columns + 1 label column (35 total)
column_names = [f"feat_{i}" for i in range(34)] + ["target"]

df = pd.read_csv(
    data_path,
    header=None,
    names=column_names,
    na_values="?"
)

# Handle missing values (e.g., age) by filling with the column median
df = df.fillna(df.median(numeric_only=True))

# ---------------------------------------------------
# 3. Prepare features and labels
# ---------------------------------------------------
X = df.drop("target", axis=1).values.astype(np.float32)
y_raw = df["target"].values.astype(int)

# Original labels are 1..6, convert to 0..5
y = y_raw - 1
num_classes = len(np.unique(y))

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=config["test_size"],
    random_state=config["random_state"],
    stratify=y
)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ---------------------------------------------------
# 4. Build Keras MLP model
# ---------------------------------------------------
def build_model(input_dim, num_classes, cfg):
    model = keras.Sequential()
    model.add(layers.Input(shape=(input_dim,)))

    # Hidden layer 1
    model.add(layers.Dense(cfg["hidden_units"][0], activation="relu"))
    model.add(layers.Dropout(cfg["dropout_rate"]))

    # Hidden layer 2
    model.add(layers.Dense(cfg["hidden_units"][1], activation="relu"))
    model.add(layers.Dropout(cfg["dropout_rate"]))

    # Output layer
    model.add(layers.Dense(num_classes, activation="softmax"))

    optimizer = keras.optimizers.Adam(learning_rate=cfg["learning_rate"])
    model.compile(
        optimizer=optimizer,
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

model = build_model(
    input_dim=X_train_scaled.shape[1],
    num_classes=num_classes,
    cfg=config
)

model.summary()

# ---------------------------------------------------
# 5. Train the model with W&B logging
# ---------------------------------------------------
callbacks = [
    WandbCallback(monitor="val_accuracy", save_weights_only=False),
    keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=8,
        restore_best_weights=True
    )
]

history = model.fit(
    X_train_scaled,
    y_train,
    validation_split=0.2,
    epochs=config["epochs"],
    batch_size=config["batch_size"],
    callbacks=callbacks,
    verbose=1
)

# ---------------------------------------------------
# 6. Evaluate and log metrics
# ---------------------------------------------------
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test, verbose=0)
error_rate = 1.0 - test_accuracy
print(f"Test accuracy: {test_accuracy:.4f}")
print(f"Test error rate: {error_rate:.4f}")

run.summary["test_accuracy"] = test_accuracy
run.summary["error_rate"] = error_rate

# Predictions for confusion matrix
y_proba = model.predict(X_test_scaled)
y_pred = np.argmax(y_proba, axis=1)

# Log confusion matrix to W&B
wandb.sklearn.plot_confusion_matrix(
    y_test,
    y_pred,
    labels=list(range(num_classes))
)

run.finish()
